## 📦 Load Required Modules

### Import necessary libraries, model config, and utility functions

In [1]:
import torch
import tiktoken
from utils.utils import generate_text_simple, token_ids_to_text, text_to_token_ids, generate
from GPT.GPT import GPTModel
from GPT.GPT_CONFIG import GPT_CONFIG_124M

## 🔤 Tokenizer Setup and Random Seed

### Initialize GPT-2 tokenizer and set a random seed for reproducibility

In [5]:
tokenizer = tiktoken.get_encoding("gpt2")
torch.manual_seed(123)

## 🚀 Set Device (CPU or GPU)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 🧠 Load Custom GPT Model & Weights

### Instantiate the GPT model and load previously trained weights from `model_local.pth`

In [3]:
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model_local.pth", map_location=device))
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

## ✍️ Generate Text from Custom GPT Model

### Use the loaded model to generate text from a prompt

In [6]:
token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=25,
    temperature=1.4
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves youlit," she up surprise. It is to face watching me by his knees
